In [119]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import json
from networkx.readwrite import json_graph

In [114]:
import random

def generate_random_connected_graph(n, p=0.4, max_tries=1000):
    """
    Generates a single random connected graph with n nodes 
    under the G(n, p) Erdős–Rényi model, retrying up to max_tries times.
    Returns:
        G (networkx.Graph): A connected graph
    Raises:
        ValueError: If unable to find a connected graph within max_tries.
    """
    for _ in range(max_tries):
        G = nx.erdos_renyi_graph(n, p)
        if nx.is_connected(G):
            return G
    raise ValueError(f"Could not generate a connected graph with n={n}, p={p} in {max_tries} tries.")


def generate_distinct_connected_graphs(
    min_nodes, 
    max_nodes, 
    graphs_per_n=2, 
    p=0.5
):
    """
    Generates random, distinct, connected graphs for each n in [min_nodes, max_nodes].
    
    Args:
        min_nodes (int): Minimum number of nodes.
        max_nodes (int): Maximum number of nodes.
        graphs_per_n (int): Number of distinct graphs to generate for each n.
        p (float): Probability of an edge (Erdős–Rényi G(n, p)).
    
    Returns:
        list of networkx.Graph: A list of generated connected graphs.
    """
    all_graphs = []
    seen_signatures = set()

    # Iterate over each node count
    for n in range(min_nodes, max_nodes + 1):
        count = 0
        tries = graphs_per_n * 2
        while count < graphs_per_n and tries > 0:
            # Generate one connected graph for size n
            G = generate_random_connected_graph(n, p=p)

            # Convert adjacency to a signature to check distinctness
            # We'll use numpy to convert adjacency matrix to a 'bytes' signature
            adj_matrix = nx.to_numpy_array(G, dtype=int)
            signature = adj_matrix.tobytes()

            if signature not in seen_signatures:
                seen_signatures.add(signature)
                all_graphs.append(G)
                count += 1
            tries -= 1

    return all_graphs

In [120]:
def save_graph_list_to_json(graphs, filename):
    """
    Saves multiple NetworkX graphs to a single JSON file, each in node-link format.
    
    Args:
        graphs (list of networkx.Graph): The graphs to save.
        filename (str): Path to the output JSON file.
    """
    data_list = []
    for G in graphs:
        graph_data = json_graph.node_link_data(G)
        data_list.append(graph_data)
    
    with open(filename, 'w') as f:
        json.dump(data_list, f, indent=2)

In [117]:
    x = 2
    y = 15
    p = 0.6
    
    distinct_graphs = generate_distinct_connected_graphs(
        min_nodes=x, 
        max_nodes=y, 
        graphs_per_n=500, 
        p=p
    )

In [121]:
PATH = "graphs.json"

save_graph_list_to_json(distinct_graphs, PATH)

/Users/linusjern/Uni/Dippa/code/local/.venv/lib/python3.12/site-packages/networkx/readwrite/json_graph/node_link.py:142: FutureWarning: 
The default value will be `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_data(G, edges="links") to preserve current behavior, or
  nx.node_link_data(G, edges="edges") for forward compatibility.
  warnings.warn(
